In [1]:
import numpy as np
import pandas as pd

### Compass data download from Smartsheet

In [2]:
excel_file_path_sfdc_version = r'C:\Users\phsheari\Documents\SFDC Data\PIPELINE UPDATE\SFDC_Trackerfile.pkl'

In [3]:
df = pd.read_pickle(excel_file_path_sfdc_version)

In [4]:
# df.head(8)

#### Setup your functions for later use

In [5]:
#A function to stringify the various possible elements in the dataframe with a float data-type
def stringify_element(m):
    a = pd.NA
    if pd.isnull(m):  # if did is null or nan
        return pd.NA
    elif type(m) == np.float64:
        return str(int(m))
    elif type(m) == float:
        return str(int(m))
    elif type(m) == np.int64:
        return str(m)
    elif type(m) == int:
        return str(m)
    elif type(m) == str:
        try:
            a = m.replace('"','').replace('&',',').replace('(','').replace(')','').replace(':',',').replace(' ',',').replace('\n',',').replace('\\n',',').replace('\\n\\r',',') .replace(',,',',').strip().split(',') 
            return a
        except:
            import pdb; pdb.set_trace()  
    else:
        return m

In [6]:
 def convert_nan_to_int(dataframe_cell):
        if pd.isnull(dataframe_cell):
            return pd.NA
        else:
            try:
                return int(dataframe_cell)
            except:
                return pd.NA

## dealdf is all the permutations of valid Deal IDs and Request IDs

In [7]:
dealdf = df[(df['Deal ID']!= -44444) & (df['Deal ID'].notnull())][['Request ID','Deal ID']]

In [8]:
dealdf

,Request ID,Deal ID
449,116418,55099391.0
458,114641,55998002.0
459,115192,55997921.0
460,115177,55997923.0
461,115168,55997925.0
...,...,...
42568,118488,44874879.0
42569,118489,55356242.0
42593,118516,56056031.0
42597,118520,17419157.0


In [9]:
dealdf['Deal ID'] = dealdf['Deal ID'].apply(stringify_element)

In [10]:
dealdf.set_index('Request ID', inplace=True)

### Explode & Clean the Deal ID Dataset here

In [11]:
dealdf = dealdf.explode('Deal ID')

In [12]:
dealdf.reset_index(inplace=True)

In [13]:
dealdf.dropna(inplace=True)

In [14]:
dealdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23467 entries, 0 to 23466
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Request ID  23467 non-null  object
 1   Deal ID     23467 non-null  object
dtypes: object(2)
memory usage: 550.0+ KB


In [15]:
deallist = []
for i,did in dealdf['Deal ID'].items():
    if isinstance(did,list):
        for e in did:
            if (pd.notnull(e) or e != ''):
                if e.isdigit():
                    deallist.append((i,e))
    else:
        deallist.append((i,did))

In [16]:
dealdf['BADDealID'] = dealdf['Deal ID'].str.isalpha()

In [17]:
dealdf['BADLENGTH'] = dealdf['Deal ID'].apply(lambda x: len(x) <= 7 or len(x) > 9)

In [18]:
dealdf.loc[dealdf['BADLENGTH']==True]

,Request ID,Deal ID,BADDealID,BADLENGTH
439,107906,44444,False,True
440,107980,44444,False,True
441,108062,44444,False,True
442,108103,44444,False,True
896,108723,99999,False,True
...,...,...,...,...
22798,104136,99999,False,True
22800,104148,99999,False,True
22849,103775,99999,False,True
22850,103809,99999,False,True


In [19]:
dealdf.drop(dealdf.loc[dealdf['BADDealID']==True].index, inplace=True)

In [20]:
dealdf.drop(dealdf.loc[dealdf['BADLENGTH']==True].index, inplace=True)

In [21]:
dealdf.drop(dealdf.loc[~(dealdf['Deal ID'].str.isdigit())].index, inplace=True)

In [22]:
dealdf.drop(dealdf.loc[dealdf['Deal ID'].isin(['12121212','12341234','321321321'])].index, inplace=True)

In [23]:
dealdf.loc[dealdf['Deal ID'].isin(['12121212','12341234'])]

,Request ID,Deal ID,BADDealID,BADLENGTH


In [24]:
dealdf.drop(columns = ['BADDealID','BADLENGTH'], inplace=True)

In [25]:
# dealdf['Deal ID'] = dealdf['Deal ID'].astype(int)

In [26]:
dealdf

,Request ID,Deal ID
0,116418,55099391
1,114641,55998002
2,115192,55997921
3,115177,55997923
4,115168,55997925
...,...,...
23462,118488,44874879
23463,118489,55356242
23464,118516,56056031
23465,118520,17419157


### Get the Cleaned Deal ID Dataset to a User-File

In [27]:
dealids = pd.DataFrame(list(dealdf['Deal ID'].unique()))

In [28]:
dealids.rename(columns={0:'Deal ID'}, inplace=True)

In [29]:
dealids.shape

(17535, 1)

In [30]:
dealids = dealids.sort_values(by='Deal ID')

In [31]:
finbideals = pd.read_pickle(r'C:\Users\phsheari\Documents\FIN BI Data\FINBI_Bookings_Accrued_Results.pkl')

In [32]:
# finbideals = pd.read_excel(r'C:\Users\phsheari\Documents\FIN BI Data\FINBI_Bookings_Accrued_Results.xlsx', sheet_name='YTD Transactions')

In [33]:
finbideals = finbideals.loc[finbideals['Booked Deal ID']!=-9999]

In [34]:
finbideals = pd.DataFrame(finbideals['Booked Deal ID'].unique())

In [35]:
finbideals.rename(columns={0:'Deal ID'}, inplace=True)

In [36]:
dealids = pd.concat([dealids,finbideals])

In [37]:
dealids = pd.DataFrame(dealids['Deal ID'].unique())

In [38]:
# dealids.rename(columns={0:'Deal ID'}, inplace=True)

In [39]:
dealids.nunique()

0    22150
dtype: int64

In [40]:
dealids.shape

(22150, 1)

In [41]:
dealids.rename(columns={0:'DealID'},inplace=True)

In [42]:
dealids.to_excel(r'C:\Users\phsheari\Documents\SFDC Data\ExplodeDealIDs.xlsx', index=False)
dealdf.to_pickle(r'C:\Users\phsheari\Documents\SFDC Data\DealIDs_RequestIDs.pkl')

### ----- Space ------- Space ------

### Prepare and Clean and Explode the GUIDs from Compass

In [43]:
Done

NameError: name 'Done' is not defined

In [ ]:
gdf = df[(df['Deal ID']!= -44444) & (df['Deal ID'].notnull())][['Request ID','GUID']]

In [ ]:
gdf['GUID'] = gdf['GUID'].apply(stringify_element)

In [ ]:
gdf = gdf.explode('GUID')

In [ ]:
gdf.dropna(inplace=True)

In [ ]:
# len(guidlist)

In [ ]:
gdf = gdf.loc[~gdf['GUID'].isnull()]

In [ ]:
gdf.shape

In [ ]:
gdf['BADLENGTH'] = gdf['GUID'].apply(lambda x: len(x) > 10)

In [ ]:
gdf.loc[gdf['BADLENGTH']==True]

In [ ]:
# ~gdf['GUID'].str.isdigit()

In [ ]:
gdf['BADGUID'] = ~gdf['GUID'].str.isdigit()

In [ ]:
gdf = gdf.drop(gdf.loc[(gdf['BADGUID']==True) | (gdf['BADLENGTH']==True)].index)

In [ ]:
gdf.drop(columns=['BADGUID','BADLENGTH'], inplace=True)

In [ ]:
gdf['GUID'] = gdf['GUID'].astype(int)

In [ ]:
gdf

### Combined the Exploded Deal ID df and the Exploded GUID df

In [ ]:
compass_explodedgdf = pd.merge(gdf,dealdf, on='Request ID')

In [ ]:
compass_explodedgdf.info()

### Prepare the SFDC Deal ID Data File

In [ ]:
# sfdc = pd.read_excel(r'C:\Users\phsheari\Documents\SFDC Data\Exports from SFDC\SFDC Extract.xlsx')

In [ ]:
sfdc.columns

In [ ]:
# sfdc = sfdc.loc[sfdc['Service Type'].isin(['AS','TS'])]

In [ ]:
sfdc.rename(columns={'Account Name':'Customer Name',
                     'Opportunity ID 18':'OpportunityID18',
                     'GU ID':'GUID',
                     'HQ Party ID':'HQPartyID',
                     'Party ID':'PartyID',
                     'Stage':'Forecast Stage',
                     'Technology & Service Name':'Technology',
                     'Opportunity Line Value ($000s) (converted)':'Service Dollars'
                     }, inplace=True)

In [ ]:
sfdc['GUID'] = sfdc['GUID'].apply(lambda x: convert_nan_to_int(x))

In [ ]:
sfdc['HQPartyID'] = sfdc['HQPartyID'].apply(lambda x: convert_nan_to_int(x))

In [ ]:
sfdc['PartyID'] = sfdc['PartyID'].apply(lambda x: convert_nan_to_int(x))

In [ ]:
sfdc = sfdc[['Customer Name','GUID','HQPartyID','PartyID','Deal ID', 'Fiscal Year','Fiscal Period','Service Source','Level 01 Territory','Level 02 Territory','Level 03 Territory','Level 04 Territory','OpportunityID18','Opportunity Status','Business Entity','Forecast Stage','Service Dollars','Services','Technology']]

In [ ]:
sfdc = sfdc.loc[(sfdc['Business Entity'].isnull()) | (sfdc['Business Entity'].isin(['AS-S','AS-T','TS-S']))]

In [ ]:
# sfdc

In [ ]:
sfdcgby = sfdc.groupby(['Customer Name', 'GUID','HQPartyID','PartyID','Deal ID','Fiscal Year','Fiscal Period','Service Source','Level 01 Territory','Level 02 Territory','Level 03 Territory','Level 04 Territory','OpportunityID18','Opportunity Status','Forecast Stage'])

In [ ]:
sfdcdeals = pd.DataFrame(sfdcgby[['Service Dollars']].sum()).reset_index()

In [ ]:
sfdcdeals = sfdcdeals.loc[sfdcdeals['Service Dollars']> 0]

In [ ]:
sfdcdeals

In [ ]:
merged = pd.merge(compass_explodedgdf,sfdcdeals, on=['GUID','Deal ID'])

In [ ]:
merged.sort_values(by=['Deal ID','Request ID'], inplace=True)

In [ ]:
merged.drop_duplicates(keep='first', inplace=True)

In [ ]:
merged['Dupechk'] = merged.duplicated(['Deal ID'], keep='first')

In [ ]:
merged.drop(merged.loc[merged['Dupechk']==True].index, inplace=True)

In [ ]:
stages = ['1 - Prospect','2 - Qualification','3 - Proposal','4 - Agreement']

In [ ]:
pd.pivot_table((merged.loc[(merged['Fiscal Year']==2021) & (merged['Forecast Stage'].isin(stages))]), values=['Service Dollars'], columns=['Forecast Stage'], index=['Fiscal Period','Level 01 Territory'], aggfunc='sum', margins=True, margins_name = 'Totals')

In [ ]:
merged.to_excel(r'C:\Users\phsheari\Documents\SFDC Data\ExplodeDealIDs_Merged.xlsx', index=False)